In [2]:
import networkx as nx
import pennylane as qml
import numpy as np

In [6]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx

# --- 1. Setup the Problem (Max-Cut) ---
# We define a simple graph: a square with 4 nodes
# (0)-(1)
#  |   |
# (3)-(2)
n_qubits = 18
graph = nx.cycle_graph(n_qubits)

# The Cost Hamiltonian H_C counts the number of uncut edges.
# Max-Cut maximizes edges where spins are different (01 or 10).
# H_C = 0.5 * sum_{(i,j) in E} (I - Z_i Z_j)
# We usually minimize energy, so we use H_C = sum Z_i Z_j.
coeffs = []
observables = []
for u, v in graph.edges():
    coeffs.append(1.0)
    observables.append(qml.PauliZ(u) @ qml.PauliZ(v))

H_cost = qml.Hamiltonian(coeffs, observables)

# The Mixer Hamiltonian H_M = sum X_i
mixer_coeffs = [1.0] * n_qubits
mixer_ops = [qml.PauliX(i) for i in range(n_qubits)]
H_mixer = qml.Hamiltonian(mixer_coeffs, mixer_ops)

# --- 2. Define the Circuit ---
# Use the standard qubit simulator
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qaoa_circuit(params, layers=1):
    # params is a flat list: [gamma_0, beta_0, gamma_1, beta_1, ...]
    
    # Initialize in superposition |+>
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    # Apply p layers
    for p in range(layers):
        gamma = params[2 * p]
        beta = params[2 * p + 1]
        
        # Cost Unitary: e^(-i gamma H_C)
        qml.ApproxTimeEvolution(H_cost, gamma, 1)
        
        # Mixer Unitary: e^(-i beta H_M)
        qml.ApproxTimeEvolution(H_mixer, beta, 1)
        
    return qml.expval(H_cost)

# --- 3. Optimization ---
layers = 5
init_params = np.random.uniform(0, np.pi, 2 * layers, requires_grad=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)

params = init_params
print(f"Initial Cost: {qaoa_circuit(params, layers=layers):.4f}")

for i in range(20):
    params = optimizer.step(lambda p: qaoa_circuit(p, layers=layers), params)
    if i % 5 == 0:
        cost = qaoa_circuit(params, layers=layers)
        print(f"Step {i}: Cost = {cost:.4f}")

print(f"Final Cost: {qaoa_circuit(params, layers=layers):.4f}")

Initial Cost: 11.7669
Step 0: Cost = 0.4808
Step 5: Cost = 1.9920
Step 10: Cost = 3.2479
Step 15: Cost = 6.6893
Final Cost: 1.7005


In [9]:
import matplotlib.pyplot as plt

# --- Drawing the Circuit ---

# 1. Define dummy parameters for the drawing
# (We need 2 * layers parameters)
dummy_params = np.array([0.1, 0.2, 0.3, 0.4]) 
layers_to_draw = 2

# 2. Use qml.draw_mpl to create the figure
# The function returns a (figure, axes) tuple
print("Drawing circuit...")
# fig, ax = qml.draw_mpl(qaoa_circuit, style='pennylane')(dummy_params, layers=layers_to_draw)

# 3. Display the plot
# plt.show()

Drawing circuit...


In [12]:
# --- 1. Setup (Same as before) ---
n_qubits = 8
graph = nx.cycle_graph(n_qubits)

coeffs = []
observables = []
for u, v in graph.edges():
    coeffs.append(1.0)
    observables.append(qml.PauliZ(u) @ qml.PauliZ(v))
H_cost = qml.Hamiltonian(coeffs, observables)

mixer_coeffs = [1.0] * n_qubits
mixer_ops = [qml.PauliX(i) for i in range(n_qubits)]
H_mixer = qml.Hamiltonian(mixer_coeffs, mixer_ops)

# --- 2. Define the DISSIPATIVE Circuit ---
# Crucial: Use 'default.mixed' to simulate density matrices
dev = qml.device("default.mixed", wires=n_qubits)

@qml.qnode(dev)
def dissipative_qaoa_circuit(params, layers=1):
    # params is now: [gamma_0, beta_0, noise_0, gamma_1, ...]
    # We added a 3rd parameter per layer for the noise strength
    
    # Initialize in superposition |+>
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    for p in range(layers):
        gamma = params[3 * p]
        beta = params[3 * p + 1]
        noise_strength = params[3 * p + 2] # Parameterized dissipation
        
        # 1. Cost Unitary
        qml.ApproxTimeEvolution(H_cost, gamma, 1)
        
        # 2. Engineered Dissipation Layer (Cooling)
        # We apply Amplitude Damping to every qubit.
        # This pushes the state locally towards |0>.
        # We constrain the noise param to be physical [0, 1] using sigmoid later, 
        # or just assume the optimizer handles it.
        for i in range(n_qubits):
            qml.AmplitudeDamping(noise_strength, wires=i)

        # 3. Mixer Unitary
        qml.ApproxTimeEvolution(H_mixer, beta, 1)
        
        # Optional: Another dissipation layer could go here
        
    return qml.expval(H_cost)

# --- 3. Optimization ---
layers = 2
# 3 parameters per layer (gamma, beta, noise)
init_params = np.random.uniform(0.01, 0.1, 3 * layers, requires_grad=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.05)

# Constraint helper to keep noise physical (0 <= p <= 1)
def constrained_cost(p):
    # Clip noise params (every 3rd param) to be between 0 and 1
    # Note: In a real rigorous optimzier, we'd use a constrained method.
    # For simple GD, we just pass the params and hope or clip inside the function.
    # Here we assume small steps keep us in valid range for the demo.
    return dissipative_qaoa_circuit(p, layers=layers)

params = init_params
print(f"Initial Cost (Dissipative): {constrained_cost(params):.4f}")

for i in range(20):
    params = optimizer.step(constrained_cost, params)
    # Simple clipping to keep probability physical
    params.numpy()[2::3] = np.clip(params.numpy()[2::3], 0, 1)
    
    if i % 5 == 0:
        cost = constrained_cost(params)
        print(f"Step {i}: Cost = {cost:.4f}")

print(f"Final Cost: {constrained_cost(params):.4f}")

Initial Cost (Dissipative): 0.6173
Step 0: Cost = 2.9612
Step 5: Cost = -1.2509
Step 10: Cost = -2.3050
Step 15: Cost = 0.5062
Final Cost: 0.0286
